In [108]:
import json # Импортируем модуль json
from pprint import pprint # Импортируем функцию pprint()
import pandas as pd # Импортируем модуль pandas
import re

with open("data.json", encoding='utf-8', newline='') as f:  # Открываем файл и связываем его с объектом "f"
    news = json.load(f)  # Загружаем содержимое открытого файла в переменную news
    
#pprint(news) # Выводим на экран содержимое переменной recipes, используя функцию pprint()

In [109]:
with open("data.json", encoding='utf-8', newline='') as f:  # Открываем файл и связываем его с объектом "f"
    news_list = json.load(f)   
n = 0    
for i in news_list:
    n +=1
print('Количество новостей в базе:', n)

Количество новостей в базе: 6164


In [110]:
with open("data.json", encoding='utf-8', newline='') as f:  # Открываем файл и связываем его с объектом "f"
    news = json.load(f) 
data = pd.DataFrame(news_list) # Создаём объект DataFrame из списка news
df = data.copy()
df = df.T.reset_index()
# Переименуем столбцы с одинакомыми названиями
df = df.rename(columns={'preview': 'contents', 'date': 'time_post', 'index': 'link'})
display(df.head()) # Выводим на экран первые строки полученного DataFrame

,link,time_post,title_post,views,contents
0,https://www.mk.ru/economics/2020/11/07/pobeda-...,2020/11/07,Победа Байдена на выборах в США может принести...,195719,Объявление результатов выборов в США затянулос...
1,https://www.mk.ru/economics/2020/11/07/rossiya...,2020/11/07,Россиян предупредили о новом необычном способе...,34114,Мошенники решили воспользоваться жаждой россия...
2,https://www.mk.ru/economics/2020/11/08/sezon-v...,2020/11/08,"Сезон в Сочи продолжается: отели забиты, о пан...",14729,"Ноябрь, а в Краснодарском крае продолжается ту..."
3,https://www.mk.ru/economics/2020/11/08/rubl-ob...,2020/11/08,Рубль обрекли на обесценивание,94359,"Похоже, «фактор Байдена» все же отразился на р..."
4,https://www.mk.ru/economics/2020/11/08/koronav...,2020/11/08,Коронавирус запустил мировую эпоху беспорядка,6839,Кругом коронавирус правит бал. С одной стороны...


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6164 entries, 0 to 6163
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   link        6164 non-null   object
 1   time_post   6164 non-null   object
 2   title_post  6164 non-null   object
 3   views       6164 non-null   object
 4   contents    6164 non-null   object
dtypes: object(5)
memory usage: 240.9+ KB


In [112]:
# Убираем рекламные новости с сайта
df = df[~df['title_post'].str.contains('1С')]
df = df[~df['contents'].str.contains('1С')]
df = df[~df['title_post'].str.contains('Выпущена версия')]
df.head()

,link,time_post,title_post,views,contents
0,https://www.mk.ru/economics/2020/11/07/pobeda-...,2020/11/07,Победа Байдена на выборах в США может принести...,195719,Объявление результатов выборов в США затянулос...
1,https://www.mk.ru/economics/2020/11/07/rossiya...,2020/11/07,Россиян предупредили о новом необычном способе...,34114,Мошенники решили воспользоваться жаждой россия...
2,https://www.mk.ru/economics/2020/11/08/sezon-v...,2020/11/08,"Сезон в Сочи продолжается: отели забиты, о пан...",14729,"Ноябрь, а в Краснодарском крае продолжается ту..."
3,https://www.mk.ru/economics/2020/11/08/rubl-ob...,2020/11/08,Рубль обрекли на обесценивание,94359,"Похоже, «фактор Байдена» все же отразился на р..."
4,https://www.mk.ru/economics/2020/11/08/koronav...,2020/11/08,Коронавирус запустил мировую эпоху беспорядка,6839,Кругом коронавирус правит бал. С одной стороны...


In [113]:
# Подгружаем ключевые слова для роли Бухгалтер
top_buh = pd.read_excel('Words.xlsx', sheet_name='Руководитель')
top_buh = top_buh['Ключевые слова'].tolist()

In [114]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language="russian")

In [115]:
# Преобразуем текст в список слов, убирая знаки препинания, а также окончания
def getwords(text):
    words = []
    for split_str in text.split():
        split_str = morph.parse(split_str)[0].normal_form
        split_str = snowball.stem(split_str)
        t = re.split("[\s;:\-_*\".,?!()'&#«»]", split_str)     
        words += [a for a in t if a != '' and len(a) > 2]
    return words

In [116]:
# Создаем столбец со списком слов:
df['words'] = df['contents'].apply(getwords)
df.head()

,link,time_post,title_post,views,contents,words
0,https://www.mk.ru/economics/2020/11/07/pobeda-...,2020/11/07,Победа Байдена на выборах в США может принести...,195719,Объявление результатов выборов в США затянулос...,"[объявлен, результат, выбор, сша, затянулось, ..."
1,https://www.mk.ru/economics/2020/11/07/rossiya...,2020/11/07,Россиян предупредили о новом необычном способе...,34114,Мошенники решили воспользоваться жаждой россия...,"[мошенник, реш, воспользова, жажд, россиянин, ..."
2,https://www.mk.ru/economics/2020/11/08/sezon-v...,2020/11/08,"Сезон в Сочи продолжается: отели забиты, о пан...",14729,"Ноябрь, а в Краснодарском крае продолжается ту...","[ноябрь, краснодарск, кра, продолжа, туристиче..."
3,https://www.mk.ru/economics/2020/11/08/rubl-ob...,2020/11/08,Рубль обрекли на обесценивание,94359,"Похоже, «фактор Байдена» все же отразился на р...","[похоже, фактор, байдена, все, отраз, российск..."
4,https://www.mk.ru/economics/2020/11/08/koronav...,2020/11/08,Коронавирус запустил мировую эпоху беспорядка,6839,Кругом коронавирус правит бал. С одной стороны...,"[круг, коронавирус, прав, бал, один, стороны, ..."


In [117]:
# Определяем ключевые слова для роли, которые присутствуют в тексте новости
def tops(text): # Определяем имя функции и передаваемые аргументы
    n = []
    for word in top_buh:
        if word.lower() in text: 
            n.append(word)
    return n # возвращаем слово

In [118]:
# Преобразуем столбец к datetime
df['time_post'] =  pd.to_datetime(df['time_post'], dayfirst=True);  # , format="%d.%m.%Y"

In [119]:
# Добавляем столбцы с ключевыми словами и количеством ключевых слов в тексте
#n = []for word in top: # Последовательно перебираем 
df['top'] = df['words'].apply(tops) # В DataFrame cоздаём столбец 
df["count"] = df["top"].apply(lambda x: len(x))

df = df.sort_values('count', ascending=False)
#df

In [120]:
# Определяем ключевые слова для роли, которые присутствуют в тексте новости
def top_title(text): # Определяем имя функции и передаваемые аргументы
    n = []
    for word in top_buh:
        if word.lower() in text.lower(): 
            n.append(word)
    return n # возвращаем слово

In [121]:
df['top_title'] = df['title_post'].apply(top_title) # В DataFrame cоздаём столбец 
df["count_title"] = df["top_title"].apply(lambda x: len(x))

In [122]:
stop = pd.read_excel('Stop_words.xlsx')
stop = stop['Ключевые слова'].tolist()

In [123]:
def stop_func(text): # Определяем имя функции и передаваемые аргументы
    n = []
    for word in stop:
        if word.lower() in text.lower(): 
            n.append(word)
    return n # возвращаем слово

In [130]:
df['stop'] = df['title_post'].apply(stop_func) # В DataFrame cоздаём столбец 
df["count_stop"] = df["stop"].apply(lambda x: len(x))
df['rank'] = df["count_title"] + df["count"] - df["count_stop"]

df = df.sort_values('rank', ascending=False)
df.head()

,link,time_post,title_post,views,contents,words,top,count,top_title,count_title,stop,count_stop,rank
1108,https://www.mk.ru/economics/2021/03/21/predpri...,2021-03-21,Предприниматель предложил оставить в покое мал...,16218,В 2020 году пандемия поставила на паузу многие...,"[2020, год, пандем, постав, пауз, мног, отрасл...",[МСП],1,[Малый бизнес],1,[],0,2
1964,https://www.mk.ru/economics/2021/07/12/bolshoe...,2021-07-12,Большое банкротство малого бизнеса: какие комп...,11286,Действующие в России ковидные ограничения прод...,"[действова, росс, ковидн, ограничен, продолжа,...",[МСП],1,[Малого бизнеса],1,[],0,2
4861,https://www.mk.ru/economics/2022/06/06/malyy-b...,2022-06-06,Малый бизнес в России показал чудеса живучести,5218,Ситуация на российском рынке кредитования малы...,"[ситуац, российск, рынок, кредитован, мал, сре...",[МСП],1,[Малый бизнес],1,[],0,2
5623,https://www.mk.ru/economics/2022/08/15/spaseni...,2022-08-15,Спасение в кризис: эксперт дал универсальные с...,4386,После начала спецоперации на Украине Россия ст...,"[посл, нача, спецоперац, украин, росс, столкну...",[МСП],1,[],0,[],0,1
5393,https://www.mk.ru/economics/2022/07/21/pustoe-...,2022-07-21,«Пустое место» МСП,5872,"Анастасия Татулова, экс-маркетолог, основавшая...","[анастас, татулова, экс, маркетолог, основа, с...",[],0,[МСП],1,[],0,1


In [125]:
from datetime import datetime, timedelta
yesterday = datetime.now() - timedelta(days=1)

yesterday.date() 

datetime.date(2022, 10, 8)

In [126]:
df[df['time_post'] == "2022-10-07"] 

,link,time_post,title_post,views,contents,words,top,count,top_title,count_title,stop,count_stop
6154,https://www.mk.ru/economics/2022/10/07/raskryt...,2022-10-07,Раскрыта мрачная тайна молодой картошки по 100...,2762,"Как и полагается в это время года, осень одари...","[как, полага, врем, года, осен, одарива, сво, ...",[],0,[],0,[],0
6152,https://www.mk.ru/economics/2022/10/07/rossiya...,2022-10-07,Россиянки «на стрессе» стали вдвое чаще поку...,1496,Объявление частичной мобилизации 21 сентября 2...,"[объявлен, частичн, мобилизац, сентябр, 2022, ...",[],0,[],0,[],0
6155,https://www.mk.ru/economics/2022/10/07/v-dume-...,2022-10-07,В Думе предложили сократить рабочую неделю для...,1049,Депутаты Госдумы выступили с идеей сократить р...,"[депутат, госдум, выступ, иде, сократ, рабоч, ...",[],0,[],0,[],0
6156,https://www.mk.ru/economics/2022/10/07/minstro...,2022-10-07,Минстрой запланировал полный запрет устанавлив...,8819,Свести к нулю риски взрывов бытового газа в кв...,"[свест, нул, риск, взрыв, бытов, газ, квартир,...",[],0,[],0,[],0
6157,https://www.mk.ru/economics/2022/10/07/khorvat...,2022-10-07,Хорватский политик дал оценку эффективности но...,1123,6 октября Евросоюз принял восьмой пакет антиро...,"[октябр, евросоюз, приня, восьм, пакет, антиро...",[],0,[],0,[Политик],1
6158,https://www.mk.ru/economics/2022/10/07/rukovod...,2022-10-07,Руководителя «Российских космических систем» с...,1741,Сменить главу акционерного общества «Российски...,"[смен, глав, акционерн, обществ, российск, кос...",[],0,[],0,[],0
6159,https://www.mk.ru/economics/2022/10/07/shvedsk...,2022-10-07,Шведская полиция объявила о «находках» на мест...,113291,Одна из главных политико-экономических интриг ...,"[один, главн, политико, экономическ, интриг, п...",[],0,[],0,[],0
6160,https://www.mk.ru/economics/2022/10/07/v-rossi...,2022-10-07,В России резко подорожал сыр,4396,,[],[],0,[],0,[],0
6161,https://www.mk.ru/economics/2022/10/07/rossiya...,2022-10-07,Россия столкнулась с невозможностью отремонтир...,11514,,[],[],0,[],0,[],0
6162,https://www.mk.ru/economics/2022/10/07/raskryt...,2022-10-07,Раскрыто будущее вторичного жилья в Москве,4911,,[],[],0,[],0,[],0


In [127]:
# Выводим вчерашние ранжированные новости в количестве 3-х штук
news = df[df['time_post'] == "2022-10-07"]['link'][:3]
news

6154    https://www.mk.ru/economics/2022/10/07/raskryt...
6152    https://www.mk.ru/economics/2022/10/07/rossiya...
6155    https://www.mk.ru/economics/2022/10/07/v-dume-...
Name: link, dtype: object